## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

Введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [81]:
# !wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [82]:
# pip install gensim

In [83]:
# pip install scipy==1.10.1

In [84]:
# pip install tqdm

In [85]:
# pip install pandas

In [86]:
# pip install ipywidgets jupyterlab_widgets widgetsnbextension

In [87]:
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [88]:
# pip install scikit-learn

In [89]:
from gensim.models.keyedvectors import KeyedVectors

In [90]:
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [4]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [5]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [6]:
import numpy as np
from numpy.linalg import norm
from tqdm.notebook import tqdm
import pandas as pd

In [7]:
# method most_simmilar
def cosine_sim(v1, v2):
    return np.array(v1@v2.T / norm(v1) / norm(v2))

In [8]:
similarity = []
indexes = []
for idx in tqdm(wv_embeddings.index_to_key):
    idx_emb = wv_embeddings[idx]
    similarity.append(cosine_sim(idx_emb, wv_embeddings['dog']))
    indexes.append(idx)
df = pd.DataFrame(
    similarity,
    index=indexes,
    columns=['similarity']).sort_values(by='similarity', ascending=False)

  0%|          | 0/1787145 [00:00<?, ?it/s]

In [9]:
df.to_csv('similarity_dog.csv')

In [10]:
df = pd.read_csv('similarity_dog.csv')

In [11]:
df.head(6)

,Unnamed: 0,similarity
0,dog,1.000000
1,animal,0.856418
2,dogs,0.788087
3,mammal,0.762381
4,cats,0.762125
5,animals,0.760794


<b>Ответ 1:</b> Слово cat не входит в top-5 близких к слову dog, но в данный топ входит слово cats

In [12]:
wv_embeddings.most_similar('cat')

[('-vet', 0.7583354711532593),
 ('dog', 0.6852341890335083),
 ('/proc/version', 0.6815888285636902),
 ('feline', 0.6779966950416565),
 ('nyan', 0.6685197353363037),
 ('com/schemas/2007/categories', 0.6678528189659119),
 ('-et', 0.6623897552490234),
 ('pajamas', 0.6485012173652649),
 ('tabby', 0.6465495824813843),
 ('catcowcat', 0.64402836561203)]

##### С другой стороны в top-5 близких к слову cat входит слово dog

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, мы его пропустим. Если вопрос не содержит ни одного известного слова, то вернем нулевой вектор.

In [13]:
import re

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

In [14]:
tokenizer = MyTokenizer()

In [15]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    question = tokenizer.tokenize(question) #
    
    question_vectors = []
    for word in question:
        if word in embeddings:
            question_vectors.append(embeddings[word])
        else:
            question_vectors.append(np.zeros(dim))
    if question_vectors == []:
        return np.zeros(dim)
    question_vectors = np.array(question_vectors).mean(axis=0)
    return question_vectors

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks`?

In [91]:
round(question_to_vec('I love neural networks', wv_embeddings, tokenizer)[2], 2)

-0.96

<b>Ответ 2:</b> -0,96

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<b>Ответ 3:</b> 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислим `DCG@10`, если $rank\_q_i^{'} = 9$

In [17]:
round(1/np.log2(1 + 9) * 1, 1)

0.3

<b>Ответ 4:</b> 0,3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [18]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = np.mean([rank <= k for rank in dup_ranks])
    return hits_value    

In [19]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = np.mean([(rank <= k) / np.log2(1 + rank) for rank in dup_ranks])
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [92]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Наш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Наш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Наш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Наш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У нас должно получиться

In [21]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [93]:
# !wget https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit

In [94]:
# !unzip stackoverflow_similar_questions.zip

Считаем данные.

In [27]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [29]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [30]:
len(validation_data)

3760

Размер нескольких первых строк

In [31]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуем функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список

**[(2, c), (0, a), (1, b)]**.

In [36]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [37]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_is_vec = question_to_vec(question, embeddings, tokenizer, dim)
    candidate_is_vecs = [question_to_vec(candidate, embeddings, tokenizer, dim)
                      for candidate in candidates]
    cosine_similarities = [
        (
            i,
            candidates[i],
            cosine_similarity(
                candidate.reshape(1, -1), question_is_vec.reshape(1, -1)
            )[0]
        ) 
        for i, candidate in enumerate(candidate_is_vecs)
    ]
    cosine_similarities.sort(key=lambda x: x[2], reverse=True)
    return [(i[0], i[1]) for i in cosine_similarities]

Протестируем работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [38]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [39]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(question)
        print(*ranks, sep='\n')
        print()

converting string to list
(1, 'C# create cookie from string and send it')
(0, 'Convert Google results object (pure js) to Python object')
(2, 'How to use jQuery AJAX for an outside domain?')

Sending array via Ajax fails
(1, 'WPF- How to update the changes in list item of a list')
(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')



Для первого экперимента мы можем полностью сравнить наши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [95]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов мы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов мы получим `для эксперимента 2`

<b>Ответ 5:</b> 102

Теперь мы можем оценить качество нашего метода. Запустим следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Возьмем для validation 1000 примеров.

In [41]:
from tqdm.notebook import tqdm

In [42]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [43]:
import pickle

In [44]:
with open('wv_ranking', 'wb') as fp:
    pickle.dump(wv_ranking, fp)

In [45]:
with open ('wv_ranking', 'rb') as fp:
    wv_ranking = pickle.load(fp)

In [46]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.879
DCG@1000: 0.444 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [47]:
train_data = read_corpus('./data/train.tsv')

Улучшим качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберем размер window равный 3.

In [48]:
train_data[0]

['converting string to list',
 'Convert Google results object (pure js) to Python object']

In [49]:
len_data = set()
for i in train_data:
    len_data.add(len(i))

In [50]:
len_data

{2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 22,
 23,
 24,
 32,
 39,
 40,
 43,
 46,
 95}

In [51]:
words = []
for questions in tqdm(train_data):
    temp = []
    for question in questions:
        temp.append(tokenizer.tokenize(question))
    words.append([c for i in temp for c in i])

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [52]:
words[0]

['converting',
 'string',
 'to',
 'list',
 'Convert',
 'Google',
 'results',
 'object',
 'pure',
 'js',
 'to',
 'Python',
 'object']

In [53]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(words,      # data for model to train on
                 vector_size=200,                # embedding vector size
                 min_count=3,             # consider words that occured at least 5 times
                 window=3).wv

In [54]:
embeddings_trained[0]

array([ 0.14655508,  0.13401586,  1.2812852 , -0.3862412 , -1.4931382 ,
       -0.23512991,  0.52709836, -1.0516547 , -1.618552  , -0.6120148 ,
        1.1346965 ,  0.84530836, -1.3533063 ,  0.9493681 , -1.1706654 ,
        0.96381897,  0.13132158,  0.89281034, -0.72785264, -0.07059141,
       -0.83882105, -0.21950379,  0.82545054, -0.03775861,  1.0543708 ,
       -0.04066826,  0.3331035 , -0.69339186,  0.20062669, -1.4839451 ,
        1.0667356 ,  2.9003057 ,  0.992881  , -0.79530877,  1.9275753 ,
       -1.7454611 ,  1.2166214 , -0.52379334,  0.10875949, -1.4206609 ,
       -1.9432591 , -0.09787348, -0.5528832 , -1.0307075 , -0.3877822 ,
        1.3026282 ,  1.2682195 ,  0.2791199 ,  0.04734347,  0.22586524,
       -1.1170361 ,  0.43883163, -0.35316446,  0.46305734,  0.53634876,
        1.1462197 , -0.25928244,  0.46222493,  0.64302474,  2.8942978 ,
        1.3424782 ,  0.20278509, -0.719558  , -1.3980956 ,  0.21107996,
       -0.95744675, -2.0081391 , -0.1335252 , -0.30521867, -0.85

In [55]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [56]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.253 | Hits@   1: 0.253
DCG@   5: 0.312 | Hits@   5: 0.367
DCG@  10: 0.337 | Hits@  10: 0.446
DCG@ 100: 0.388 | Hits@ 100: 0.696
DCG@ 500: 0.415 | Hits@ 500: 0.912
DCG@1000: 0.425 | Hits@1000: 1.000


Попробуем удалить стоп-слова и лемматизировать текст перед этим

In [59]:
# !pip install nltk

In [60]:
import nltk
from nltk.tokenize import word_tokenize

In [96]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))
nltk.download('wordnet')
wnl = nltk.WordNetLemmatizer()

In [97]:
nltk.download('punkt')
for questions in tqdm(train_data):
    for question in questions:
        temp = []
        temp.append(' '.join(
            [wnl.lemmatize(word) for word in word_tokenize(question) if word not in stopWords]
        ))
    words.append([c for i in temp for c in i])

In [65]:
embeddings_trained = Word2Vec(words,      # data for model to train on
                 vector_size=200,                # embedding vector size
                 min_count=3,             # consider words that occured at least 5 times
                 window=3).wv

In [66]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [67]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.237 | Hits@   1: 0.237
DCG@   5: 0.295 | Hits@   5: 0.348
DCG@  10: 0.317 | Hits@  10: 0.418
DCG@ 100: 0.372 | Hits@ 100: 0.693
DCG@ 500: 0.401 | Hits@ 500: 0.913
DCG@1000: 0.410 | Hits@1000: 1.000


Попробуем удалить стоп-слова и не лемматизировать текст перед этим

In [68]:
stopWords = set(stopwords.words('english'))

In [69]:
preproc_words = [tokenizer.tokenize(' '.join(data).lower()) 
                 for data in train_data]
words = [[word for word in row if word not in stopWords] 
         for row in preproc_words]

In [70]:
embeddings_trained = Word2Vec(words,      # data for model to train on
                 vector_size=200,                # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=20).wv

In [71]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [72]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.284 | Hits@   1: 0.284
DCG@   5: 0.352 | Hits@   5: 0.414
DCG@  10: 0.372 | Hits@  10: 0.473
DCG@ 100: 0.416 | Hits@ 100: 0.690
DCG@ 500: 0.438 | Hits@ 500: 0.866
DCG@1000: 0.452 | Hits@1000: 1.000


Попробуем свой кастомный токенайзер

In [73]:
import string
from nltk.tokenize import WordPunctTokenizer

In [98]:
nltk.download('stopwords')
nltk.download('wordnet')
stopWords = set(stopwords.words('english'))

tokenizer = WordPunctTokenizer()
wnl = nltk.WordNetLemmatizer()

In [75]:
class CustomTokenizer:
    def tokenize(self, text):
        text = re.sub(f'[{string.punctuation}]', ' ', text)
        return [wnl.lemmatize(word) for word in tokenizer.tokenize(text.lower()) if word not in stopWords]

In [76]:
st = "Oh, I think I ve landed Where there are miracles at work,  For the thirst and for the hunger Come the conference of birds"
custom_tokenizer = CustomTokenizer()
custom_tokenizer.tokenize(st)

['oh',
 'think',
 'landed',
 'miracle',
 'work',
 'thirst',
 'hunger',
 'come',
 'conference',
 'bird']

In [77]:
words = []
for pair in tqdm(train_data):
    for sent in pair:
        words.append(custom_tokenizer.tokenize(sent))

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [78]:
embeddings_trained = Word2Vec(words, # data for model to train on
                 vector_size=200,                 # embedding vector size
                 min_count=7,             # consider words that occured at least 5 times
                 window=3).wv

In [79]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, custom_tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [80]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.396 | Hits@   1: 0.396
DCG@   5: 0.487 | Hits@   5: 0.566
DCG@  10: 0.511 | Hits@  10: 0.640
DCG@ 100: 0.558 | Hits@ 100: 0.867
DCG@ 500: 0.572 | Hits@ 500: 0.974
DCG@1000: 0.575 | Hits@1000: 1.000


С кастомным токенайзером выглядит лучше

### Выводы о полученных результатах:

* C кастомным токенайзаром результат намного лучше
* Нормализация слов помогает в улучшении качества
* Самостоятельно обученные эмбеддинги лучше справились с задачей
* Из-за простоты попробованных методов у нас получиличь не самые лучшие решения